In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

import django
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'Carkinos.settings.local'
django.setup()

from probes.models import Dataset,Platform,Sample,CellLine,ProbeID
root=Path('../').resolve()
plus2_path=root.joinpath('src','raw','Affy_U133plus2_probe_info.csv')
val_pth=Path('../').resolve().joinpath('src','PCA_TEST.quantile_normalized.tsv')

In [2]:
import sklearn
from sklearn.decomposition import PCA

In [3]:
import pylab
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [4]:
val = pd.read_table(val_pth.as_posix())
original_data=val

In [5]:
val.head()

,Unnamed: 0,GSM803616_113385hp133a11.cel.gz,GSM803621_113390hp133a11.cel.gz,GSM803633_113402hp133a11.cel.gz,GSM803636_113405hp133a11.cel.gz,GSM803640_113409hp133a11.cel.gz,GSM803641_113410hp133a11.cel.gz,GSM803642_113411hp133a11.cel.gz,GSM803643_113412hp133a11.cel.gz,GSM803648_113417hp133a11.cel.gz,...,GSM886845.CEL.gz,GSM886856.CEL.gz,GSM886858.CEL.gz,GSM886863.CEL.gz,GSM886894.CEL.gz,GSM886902.CEL.gz,GSM886940.CEL.gz,GSM886988.CEL.gz,GSM887063.CEL.gz,GSM887083.CEL.gz
0,1007_s_at,8.466508,8.704320,10.268638,10.743102,8.951665,9.458733,9.652571,9.850145,9.594955,...,8.461307,9.713824,9.263093,9.417826,8.957506,9.192340,9.659516,9.366531,9.427184,8.005455
1,1053_at,8.980473,8.877624,9.267413,8.228491,8.813888,8.575776,8.217835,7.745895,8.709977,...,9.298895,9.052348,9.157794,9.815388,9.288431,8.515813,9.575967,9.606156,9.315189,8.951114
2,117_at,6.829027,6.657378,6.710125,6.754070,6.695269,6.947095,6.628113,6.639558,6.817780,...,6.364418,6.418995,6.339765,6.465320,6.319365,6.380699,6.401607,6.329362,6.411867,6.357796
3,121_at,8.052456,8.211668,8.385423,8.079701,9.812167,9.175841,10.420003,9.760561,7.938326,...,9.438799,9.287558,7.431182,9.968309,7.775767,9.713154,7.891176,7.216398,7.488675,7.270498
4,1255_g_at,6.234672,6.050170,6.178644,6.130865,6.062958,6.040505,6.003245,6.105956,6.152480,...,6.021642,6.083411,6.121256,6.018776,6.175244,6.090674,5.948331,5.985304,6.018757,6.104131


In [6]:
val.index = val['Unnamed: 0']
val.index.name = None
val_test = val.iloc[:, 1:]
val_test.head()

,GSM803616_113385hp133a11.cel.gz,GSM803621_113390hp133a11.cel.gz,GSM803633_113402hp133a11.cel.gz,GSM803636_113405hp133a11.cel.gz,GSM803640_113409hp133a11.cel.gz,GSM803641_113410hp133a11.cel.gz,GSM803642_113411hp133a11.cel.gz,GSM803643_113412hp133a11.cel.gz,GSM803648_113417hp133a11.cel.gz,GSM803663_113432hp133a11.cel.gz,...,GSM886845.CEL.gz,GSM886856.CEL.gz,GSM886858.CEL.gz,GSM886863.CEL.gz,GSM886894.CEL.gz,GSM886902.CEL.gz,GSM886940.CEL.gz,GSM886988.CEL.gz,GSM887063.CEL.gz,GSM887083.CEL.gz
1007_s_at,8.466508,8.704320,10.268638,10.743102,8.951665,9.458733,9.652571,9.850145,9.594955,10.222705,...,8.461307,9.713824,9.263093,9.417826,8.957506,9.192340,9.659516,9.366531,9.427184,8.005455
1053_at,8.980473,8.877624,9.267413,8.228491,8.813888,8.575776,8.217835,7.745895,8.709977,8.456886,...,9.298895,9.052348,9.157794,9.815388,9.288431,8.515813,9.575967,9.606156,9.315189,8.951114
117_at,6.829027,6.657378,6.710125,6.754070,6.695269,6.947095,6.628113,6.639558,6.817780,6.746117,...,6.364418,6.418995,6.339765,6.465320,6.319365,6.380699,6.401607,6.329362,6.411867,6.357796
121_at,8.052456,8.211668,8.385423,8.079701,9.812167,9.175841,10.420003,9.760561,7.938326,7.931323,...,9.438799,9.287558,7.431182,9.968309,7.775767,9.713154,7.891176,7.216398,7.488675,7.270498
1255_g_at,6.234672,6.050170,6.178644,6.130865,6.062958,6.040505,6.003245,6.105956,6.152480,6.005181,...,6.021642,6.083411,6.121256,6.018776,6.175244,6.090674,5.948331,5.985304,6.018757,6.104131


In [7]:
new_name=["GSM886845.CEL.gz",
"GSM803640_113409hp133a11.cel.gz",
"GSM803699_113468hp133a11.cel.gz",
"GSM803758_118194hp133a11.cel.gz",
"GSM886856.CEL.gz",
"GSM803641_113410hp133a11.cel.gz",
"GSM803700_113469hp133a11.cel.gz",
"GSM803759_118195hp133a11.cel.gz",
"GSM886858.CEL.gz",
"GSM803648_113417hp133a11.cel.gz",
"GSM803707_113476hp133a11.cel.gz",
"GSM803765_118202hp133a11.cel.gz",
"GSM886863.CEL.gz",
"GSM803642_113411hp133a11.cel.gz",
"GSM803701_113470hp133a11.cel.gz",
"GSM803760_118196hp133a11.cel.gz",
"GSM886894.CEL.gz",
"GSM803621_113390hp133a11.cel.gz",
"GSM803680_113449hp133a11.cel.gz",
"GSM803739_118175hp133a11.cel.gz",
"GSM886902.CEL.gz",
"GSM803643_113412hp133a11.cel.gz",
"GSM803702_113471hp133a11.cel.gz",
"GSM886940.CEL.gz",
"GSM803633_113402hp133a11.cel.gz",
"GSM803692_113461hp133a11.cel.gz",
"GSM803751_118187hp133a11.cel.gz",
"GSM886988.CEL.gz",
"GSM803663_113432hp133a11.cel.gz",
"GSM803721_113490hp133a11.cel.gz",
"GSM803779_118217hp133a11.cel.gz",
"GSM887063.CEL.gz",
"GSM803636_113405hp133a11.cel.gz",
"GSM803695_113464hp133a11.cel.gz",
"GSM803754_118190hp133a11.cel.gz",
"GSM887083.CEL.gz",
"GSM803616_113385hp133a11.cel.gz",
"GSM803675_113444hp133a11.cel.gz",
"GSM803734_118170hp133a11.cel.gz"]

In [8]:
val=val.reindex_axis(new_name,axis=1)
val

,GSM886845.CEL.gz,GSM803640_113409hp133a11.cel.gz,GSM803699_113468hp133a11.cel.gz,GSM803758_118194hp133a11.cel.gz,GSM886856.CEL.gz,GSM803641_113410hp133a11.cel.gz,GSM803700_113469hp133a11.cel.gz,GSM803759_118195hp133a11.cel.gz,GSM886858.CEL.gz,GSM803648_113417hp133a11.cel.gz,...,GSM803721_113490hp133a11.cel.gz,GSM803779_118217hp133a11.cel.gz,GSM887063.CEL.gz,GSM803636_113405hp133a11.cel.gz,GSM803695_113464hp133a11.cel.gz,GSM803754_118190hp133a11.cel.gz,GSM887083.CEL.gz,GSM803616_113385hp133a11.cel.gz,GSM803675_113444hp133a11.cel.gz,GSM803734_118170hp133a11.cel.gz
1007_s_at,8.461307,8.951665,9.015657,9.114502,9.713824,9.458733,9.473240,9.889285,9.263093,9.594955,...,10.253357,10.218173,9.427184,10.743102,10.714761,10.652688,8.005455,8.466508,8.407569,8.465109
1053_at,9.298895,8.813888,8.754567,8.090032,9.052348,8.575776,8.510183,9.063475,9.157794,8.709977,...,8.346023,8.245170,9.315189,8.228491,8.193194,8.255571,8.951114,8.980473,8.859228,9.216713
117_at,6.364418,6.695269,6.768465,6.670836,6.418995,6.947095,6.920350,7.032093,6.339765,6.817780,...,6.776119,6.830709,6.411867,6.754070,6.701130,6.817833,6.357796,6.829027,6.825796,6.723393
121_at,9.438799,9.812167,9.768166,9.439228,9.287558,9.175841,9.283408,9.229964,7.431182,7.938326,...,7.878160,7.943019,7.488675,8.079701,8.278690,8.229162,7.270498,8.052456,8.317451,8.175203
1255_g_at,6.021642,6.062958,6.247082,6.190257,6.083411,6.040505,6.193966,6.110434,6.121256,6.152480,...,6.106823,6.060658,6.018757,6.130865,6.117855,6.117373,6.104131,6.234672,6.196147,6.092855
1294_at,7.143608,7.677207,7.858427,7.484464,7.341028,7.303248,7.340732,7.563531,6.542747,7.083726,...,7.113014,7.101001,6.670105,7.296643,7.339645,7.425366,7.409747,7.705363,7.714117,7.801424
1316_at,6.581450,6.805741,6.829712,6.774518,7.008686,6.722838,6.862099,6.792364,7.363535,6.790761,...,6.756319,6.798425,7.108618,6.756501,6.746921,6.719750,7.399729,6.678366,6.863537,6.973697
1320_at,6.473493,7.015123,6.994869,6.944735,6.657676,7.003863,6.835708,6.726457,6.817740,6.882488,...,6.633831,6.671301,6.815076,6.753532,6.654617,6.579627,6.269738,6.520533,6.659820,6.658652
1405_i_at,6.275908,6.191226,6.241302,6.212968,6.557764,7.911274,7.894463,7.317273,6.244645,6.194665,...,6.288121,6.180302,6.046728,6.157490,6.093714,6.206802,6.930427,9.010700,8.752516,6.539585
1431_at,6.277947,6.344488,6.423654,6.644586,6.227907,6.347405,6.305207,6.261263,6.219192,6.236192,...,6.268193,6.168492,6.148696,6.357650,6.256896,6.237912,6.364707,6.521723,6.590202,6.444873


In [9]:
val=val.as_matrix()
val

array([[ 8.46130748,  8.95166541,  9.01565746, ...,  8.46650755,
         8.4075694 ,  8.46510882],
       [ 9.2988951 ,  8.81388777,  8.75456687, ...,  8.98047324,
         8.85922819,  9.21671311],
       [ 6.36441766,  6.69526869,  6.7684647 , ...,  6.82902661,
         6.8257959 ,  6.72339295],
       ..., 
       [ 6.19054803,  6.05337759,  6.06055829, ...,  6.1338922 ,
         6.10259233,  6.15488734],
       [ 6.39890069,  6.39669175,  6.27705151, ...,  6.34489862,
         6.36396778,  6.41016202],
       [ 6.19425287,  6.19279484,  6.1585468 , ...,  6.20549658,
         6.21847785,  6.32206949]])

In [10]:
t_val=np.transpose(val)
pca= PCA(n_components=54675)
X = pca.fit_transform(t_val)
#print(X)

In [11]:
t=pca.explained_variance_ratio_
sum(t[0:40])

1.0000000000000004

In [12]:
for n in range(1,40):
 print('propotion ',n+1,':',sum(t[0:n]))

propotion  2 : 0.224013222551
propotion  3 : 0.348340536504
propotion  4 : 0.45032991264
propotion  5 : 0.537471099666
propotion  6 : 0.614492157415
propotion  7 : 0.682509628289
propotion  8 : 0.743164833524
propotion  9 : 0.793767503591
propotion  10 : 0.836645963709
propotion  11 : 0.867664062001
propotion  12 : 0.88729130498
propotion  13 : 0.902482804049
propotion  14 : 0.915169405053
propotion  15 : 0.926347810458
propotion  16 : 0.935407341958
propotion  17 : 0.943699256998
propotion  18 : 0.950692270153
propotion  19 : 0.957221588842
propotion  20 : 0.9633487424
propotion  21 : 0.968933758838
propotion  22 : 0.973761566473
propotion  23 : 0.978295231422
propotion  24 : 0.982027577088
propotion  25 : 0.985051174671
propotion  26 : 0.987625100781
propotion  27 : 0.989650909975
propotion  28 : 0.991106444027
propotion  29 : 0.99245149015
propotion  30 : 0.993558126158
propotion  31 : 0.994517117929
propotion  32 : 0.995446510957
propotion  33 : 0.996306236828
propotion  34 : 0.997

In [13]:
#colormap = plt.cm.jet
#colm=[colormap(1)]*29+[colormap(1000)]*10
col1=['r','b','b','b']*5+['r','b','b']+['r','b','b','b']*4
col=['r']*4+['b']*4+['g']*4+['m']*4+['k']*4+['c']*3+['y']*4+['#05f6f0']*4+['#75f605']*4+['#f472ce']*4
print(col1[0:6])
print(col[0:16])

['r', 'b', 'b', 'b', 'r', 'b']
['r', 'r', 'r', 'r', 'b', 'b', 'b', 'b', 'g', 'g', 'g', 'g', 'm', 'm', 'm', 'm']


In [14]:
#fig = pylab.figure()
#ax = Axes3D(fig)  
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(1, 2, 1, projection='3d')
xx=np.array(X[:,0]).flatten()
yy=np.array(X[:,1]).flatten()
zz=np.array(X[:,2]).flatten()
ax.scatter(xx,yy,zz,s=80,marker=(5,3),c=col[:])
ax.set_title("colored by cell line")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax = fig.add_subplot(1, 2, 2, projection='3d')
xx=np.array(X[:,0]).flatten()
yy=np.array(X[:,1]).flatten()
zz=np.array(X[:,2]).flatten()
ax.scatter(xx,yy,zz,s=80,marker=(5,3),c=col1[:])
ax.set_title("colored by data set")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show() 

In [121]:
tt=original_data.loc[['212581_x_at','213453_x_at','217398_x_at',
'AFFX-HUMGAPDH/M33197_3_at',
'AFFX-HUMGAPDH/M33197_5_at',
'AFFX-HUMGAPDH/M33197_M_at'
],:]
tt=tt.iloc[:,1:]
tt

,GSM803616_113385hp133a11.cel.gz,GSM803621_113390hp133a11.cel.gz,GSM803633_113402hp133a11.cel.gz,GSM803636_113405hp133a11.cel.gz,GSM803640_113409hp133a11.cel.gz,GSM803641_113410hp133a11.cel.gz,GSM803642_113411hp133a11.cel.gz,GSM803643_113412hp133a11.cel.gz,GSM803648_113417hp133a11.cel.gz,GSM803663_113432hp133a11.cel.gz,...,GSM886845.CEL.gz,GSM886856.CEL.gz,GSM886858.CEL.gz,GSM886863.CEL.gz,GSM886894.CEL.gz,GSM886902.CEL.gz,GSM886940.CEL.gz,GSM886988.CEL.gz,GSM887063.CEL.gz,GSM887083.CEL.gz
212581_x_at,13.692688,13.784569,13.768367,13.735409,13.945257,13.783133,13.770379,13.930343,13.981070,13.678390,...,13.744419,13.637974,13.802674,13.906091,13.591915,13.963205,13.704083,13.717885,13.686001,13.211381
213453_x_at,13.620251,13.740132,13.691431,13.690015,13.912630,13.701529,13.723103,13.911382,13.924181,13.598943,...,13.720177,13.594825,13.746064,13.868072,13.472050,13.962561,13.633952,13.698346,13.646558,13.143625
217398_x_at,13.625629,13.766562,13.755158,13.734911,13.955820,13.747659,13.723694,13.952851,13.947258,13.680103,...,13.701262,13.651801,13.822772,13.936859,13.568534,13.769095,13.722036,13.730397,13.688391,13.244282
AFFX-HUMGAPDH/M33197_3_at,13.487710,13.614178,13.666509,13.573509,13.748241,13.624083,13.647922,13.837788,13.770546,13.567317,...,13.572755,13.639457,13.659317,13.821375,13.471063,13.770787,13.592931,13.665752,13.646805,13.424041
AFFX-HUMGAPDH/M33197_5_at,13.548932,13.621462,13.697945,13.634097,13.863217,13.695005,13.741313,13.782010,13.824497,13.640025,...,13.576829,13.480508,13.685483,13.951764,13.330604,13.766827,13.653799,13.708632,13.587779,12.355801
AFFX-HUMGAPDH/M33197_M_at,13.599816,13.638201,13.713933,13.654290,13.865454,13.703310,13.746095,13.883743,13.876947,13.565019,...,13.735935,13.608367,13.690496,13.881873,13.400269,13.881500,13.467286,13.645673,13.631237,12.627046


In [128]:
means=tt.mean().as_matrix()
means

array([ 13.59583757,  13.69418395,  13.71555711,  13.67037183,
        13.88176989,  13.70911968,  13.72541767,  13.88301942,
        13.88741627,  13.62163277,  13.62041696,  13.69417411,
        13.7016469 ,  13.67529188,  13.85954428,  13.70434524,
        13.68917396,  13.87420595,  13.88528117,  13.61831592,
        13.46847072,  13.67050424,  13.66087641,  13.62801464,
        13.71286136,  13.8154521 ,  13.89340109,  13.76977795,
        13.51029685,  13.67522967,  13.60215521,  13.73446785,
        13.89433915,  13.47240563,  13.85232918,  13.62901466,
        13.69444748,  13.64779517,  13.00102936])

In [137]:
8.95166541-13.69418395

-4.742518539999999

In [135]:
val

array([[  8.46130748,   8.95166541,   9.01565746,   9.11450207,
          9.71382391,   9.45873324,   9.47324011,   9.88928544,
          9.26309348,   9.59495523,   9.66603378,   9.65130961,
          9.41782607,   9.65257113,   9.83507795,   9.61984108,
          8.95750628,   8.70432039,   8.7553945 ,   8.69213714,
          9.19234021,   9.85014465,   9.90862442,   9.65951572,
         10.26863798,  10.21440527,  10.16297405,   9.36653101,
         10.22270503,  10.25335657,  10.21817286,   9.42718364,
         10.74310215,  10.71476125,  10.65268828,   8.00545508,
          8.46650755,   8.4075694 ,   8.46510882],
       [  9.2988951 ,   8.81388777,   8.75456687,   8.09003166,
          9.05234794,   8.57577625,   8.51018326,   9.06347503,
          9.1577942 ,   8.70997655,   8.8155648 ,   8.3834526 ,
          9.81538781,   8.21783451,   7.92563044,   9.29293086,
          9.28843147,   8.87762358,   8.74875803,   8.9408213 ,
          8.51581321,   7.74589479,   7.50285323,   9

In [138]:
norm_val=val-means

In [139]:
t_norm_val=np.transpose(norm_val)
pca= PCA(n_components=3)
norm_X = pca.fit_transform(t_norm_val)
print(norm_X)

[[ 0.57831706  1.27510446  0.21220149]
 [ 1.03317342  0.71330756 -0.05002879]
 [ 0.99976985  0.62911075 -0.09226462]
 [ 0.88060922  0.17434654 -0.60546659]
 [ 0.35864886  0.1175457   0.10819232]
 [ 0.52847743  0.02409358 -0.1177726 ]
 [ 0.63520201  0.01516286 -0.16798159]
 [ 0.35829613 -0.08140018  0.24045808]
 [-1.51262059  0.18854462 -0.30530986]
 [-0.60872798 -0.29528334  0.01846387]
 [-0.37634376 -0.25226118  0.18246918]
 [-0.48602436 -0.44110487 -0.3557259 ]
 [ 0.97597917  0.80037566  1.02230836]
 [ 1.84104994 -0.0402972  -0.12018721]
 [ 2.13683897 -0.20848845 -0.506199  ]
 [ 1.42131809  0.43668209  0.74184298]
 [-1.05933913  0.53440974  0.04288752]
 [-0.71959085  0.68096125 -0.65203495]
 [-0.60872237  0.59985631 -0.72538171]
 [-0.43080784  0.66386172 -0.16217409]
 [ 1.21875903  0.31561608  0.13993411]
 [ 1.34150763 -0.55385075 -0.51563511]
 [ 1.23513133 -0.75972671 -0.70132463]
 [-0.88597171  0.04644456  0.7283155 ]
 [-0.32867346 -0.51143745  0.7276851 ]
 [-0.34388716 -0.41488408

In [140]:
norm_t=pca.explained_variance_ratio_
print('propotion:',sum(norm_t[0:2]))

propotion: 0.830936098325


In [170]:
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(1, 2, 1, projection='3d')
xx=np.array(norm_X[:,0]).flatten()
yy=np.array(norm_X[:,1]).flatten()
zz=np.array(norm_X[:,2]).flatten()
ax.scatter(xx,yy,zz,s=80,marker=(5,3),c=col[:])
ax.set_title("norm_data colored by cell line")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax = fig.add_subplot(1, 2, 2, projection='3d')
xx=np.array(norm_X[:,0]).flatten()
yy=np.array(norm_X[:,1]).flatten()
zz=np.array(norm_X[:,2]).flatten()
ax.scatter(xx,yy,zz,s=80,marker=(5,3),c=col1[:])
ax.set_title("norm_data colored by data set")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show() 

In [171]:
old_data=np.array([
[8.9283,8.8862,8.9374,9.0085,10.151,9.2932,9.2658,9.7032,9.675,9.4598,9.4706,9.4283],
[10.0503,8.6776,8.597,7.9725,9.5993,8.45,8.4428,8.9239,9.6814,8.6094,8.6629,8.3004],
[6.8854,6.6461,6.6673,6.6335,6.9292,6.879,6.8687,6.9176,6.8493,6.761,6.6705,6.6293],
[10.502,9.5326,9.41,9.1377,10.2953,8.8643,8.9176,8.8962,8.064,7.8083,7.9111,7.969],
[6.8712,5.8737,6.0063,5.963,6.7891,5.836,5.9821,5.9546,6.7754,5.986,5.9624,5.8532]
])


In [181]:
old_tval=np.transpose(old_data)
pca= PCA(n_components=3)
old_X = pca.fit_transform(old_tval)
print(old_X)

[[-2.0812633  -0.13694719  0.52126024]
 [-0.33850833  0.70599732  0.20481152]
 [-0.23874222  0.62310044  0.1748619 ]
 [ 0.28846196  0.88478263 -0.04220665]
 [-1.6752425  -0.43474881 -0.69284278]
 [ 0.324978    0.34828994 -0.11791176]
 [ 0.24128995  0.33472026 -0.08704889]
 [ 0.0481936  -0.16678024 -0.31664949]
 [ 0.1670874  -1.36811765  0.27982773]
 [ 1.10332206 -0.39145307  0.06611892]
 [ 1.00513939 -0.36328696  0.05973082]
 [ 1.15528398 -0.03555665 -0.04995157]]


In [182]:
old_norm_t=pca.explained_variance_ratio_
print('propotion:',sum(old_norm_t[0:2]))

propotion: 0.922274193103


In [183]:
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(1, 2, 1, projection='3d')
xx=np.array(old_X[:,0]).flatten()
yy=np.array(old_X[:,1]).flatten()
zz=np.array(old_X[:,2]).flatten()
ax.scatter(xx,yy,zz,s=80,marker=(5,3),c=col[:12])
ax.set_title("old_data colored by cell line")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax = fig.add_subplot(1, 2, 2, projection='3d')
xx=np.array(old_X[:,0]).flatten()
yy=np.array(old_X[:,1]).flatten()
zz=np.array(old_X[:,2]).flatten()
ax.scatter(xx,yy,zz,s=80,marker=(5,3),c=col1[:12])
ax.set_title("old_data colored by data set")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show() 

In [175]:
old_norm_data=np.array([
[-5.5208,-4.7194,-4.6519,-4.4318,-4.1799,-4.1518,-4.1751,-3.8435,-4.7586,-4.1699,-4.1486,-4.0774],
[-4.3988,-4.928,-4.9924,-5.4678,-4.7316,-4.995,-4.9981,-4.6228,-4.7522,-5.0204,-4.9562,-5.2052],
[-7.5637,-6.9595,-6.922,-6.8069,-7.4017,-6.566,-6.5722,-6.6292,-7.5843,-6.8687,-6.9487,-6.8763],
[-3.9471,-4.1793,-4.1793,-4.3026,-4.0355,-4.5807,-4.5234,-4.6506,-6.3696,-5.8215,-5.7081,-5.5366],
[-7.5779,-7.7319,-7.5813,-7.4773,-7.5417,-7.6091,-7.4589,-7.5922,-7.6582,-7.6437,-7.6567,-7.6524]
])

In [176]:
old_norm_tval=np.transpose(old_norm_data)
pca= PCA(n_components=3)
old_norm_X = pca.fit_transform(old_norm_tval)
print(old_norm_X)

[[-1.11628256  1.16608375 -0.02995322]
 [-0.68402189  0.11460089  0.1375358 ]
 [-0.6718589   0.01831752  0.16620581]
 [-0.47854629 -0.33215007  0.47019693]
 [-0.73360049  0.016997   -0.42765057]
 [-0.17671972 -0.48865285  0.01836763]
 [-0.2431788  -0.48486353  0.03133571]
 [-0.07087121 -0.55310508 -0.45662019]
 [ 1.43063435  0.9396548  -0.01883581]
 [ 1.0334845  -0.08852625  0.03985452]
 [ 0.92354147 -0.05563775 -0.05082307]
 [ 0.78741955 -0.25271843  0.12038646]]


In [177]:
old_n_t=pca.explained_variance_ratio_
print('propotion:',sum(old_n_t[0:2]))

propotion: 0.911316894186


In [180]:
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(1, 2, 1, projection='3d')
xx=np.array(old_norm_X[:,0]).flatten()
yy=np.array(old_norm_X[:,1]).flatten()
zz=np.array(old_norm_X[:,2]).flatten()
ax.scatter(xx,yy,zz,s=80,marker=(5,3),c=col[:12])
ax.set_title("old_norm_data colored by cell line")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax = fig.add_subplot(1, 2, 2, projection='3d')
xx=np.array(old_norm_X[:,0]).flatten()
yy=np.array(old_norm_X[:,1]).flatten()
zz=np.array(old_norm_X[:,2]).flatten()
ax.scatter(xx,yy,zz,s=80,marker=(5,3),c=col1[:12])
ax.set_title("old_norm_data colored by data set")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show() 

In [185]:
ppth=Path('../').resolve().joinpath('src','raw','ptest_data.xlsx')
pp=pd.read_excel(ppth.as_posix())
pp=pp.as_matrix()
pp

array([[-5.366 , -5.7631, -5.6608, -5.6256, -4.8057, -5.6661, -5.5468,
        -5.3902, -5.0451, -5.2662, -4.8927, -4.9939, -4.3508, -4.3943,
        -5.421 , -5.2474],
       [-7.1705, -7.3635, -5.2312, -5.1687, -7.5804, -6.7095, -6.1366,
        -7.5169, -6.9669, -6.0586, -7.5595, -5.2557, -7.2247, -5.8687,
        -7.0951, -6.9543],
       [-6.1506, -6.4969, -5.673 , -5.4194, -6.6505, -5.8046, -5.7578,
        -6.7381, -5.7962, -5.5539, -6.1081, -4.7379, -6.3432, -5.4007,
        -5.9156, -5.9778],
       [-7.4163, -7.6428, -6.2846, -6.0678, -7.7376, -7.6556, -6.4619,
        -7.7967, -7.8114, -6.2607, -7.6419, -5.7602, -7.5193, -6.1356,
        -7.8308, -7.8084],
       [-6.7061, -7.0073, -5.8592, -5.5943, -6.6342, -6.8321, -6.0719,
        -7.2331, -7.0227, -5.86  , -6.987 , -5.1277, -6.334 , -5.5927,
        -7.0556, -7.1553],
       [-7.2542, -7.5315, -6.0611, -6.0768, -7.5042, -6.9675, -6.3497,
        -7.2016, -6.7728, -6.1676, -7.3919, -5.5271, -7.2688, -5.9041,
        -7.07

In [187]:
old_norm_tval=np.transpose(pp)
pca= PCA(n_components=3)
old_norm_X = pca.fit_transform(old_norm_tval)
print(old_norm_X)

[[  3.57909214e-01   1.45167116e+00  -8.34760590e-01]
 [  1.43705365e+00   6.57368909e-01   1.69662377e-01]
 [ -2.14339445e+00   2.44562198e-01   5.10341660e-01]
 [ -2.55896716e+00   2.95749196e-01   3.19789509e-01]
 [  1.84064926e+00  -3.40068949e-01  -5.35856542e-01]
 [  6.97232647e-01   3.67917331e-01   4.34733141e-01]
 [ -8.78073683e-01  -4.08845839e-02   1.33606554e-01]
 [  2.12257996e+00  -5.11199088e-01   5.66596069e-01]
 [  8.08135778e-01  -2.44727149e-01   3.84347661e-01]
 [ -1.47559264e+00  -3.31625912e-01   8.32919369e-02]
 [  1.87500035e+00  -6.39713082e-01  -6.66600459e-02]
 [ -3.36681271e+00  -3.94783019e-01  -1.07724701e-01]
 [  1.18423070e+00  -8.76125244e-01  -5.41964184e-01]
 [ -2.13883587e+00  -4.43423820e-01  -7.24914269e-01]
 [  1.10211353e+00   3.52543067e-01   2.07640348e-01]
 [  1.13677141e+00   4.52738990e-01   1.87107653e-03]]


In [188]:
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(1, 2, 1, projection='3d')
xx=np.array(old_norm_X[:,0]).flatten()
yy=np.array(old_norm_X[:,1]).flatten()
zz=np.array(old_norm_X[:,2]).flatten()
ax.scatter(xx,yy,zz,s=80,marker=(5,3),c=['r','r','r','r','g','g','g','b','b','b','c','c','m','m','g','g'])
ax.set_title("ppdata colored by cell line")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax = fig.add_subplot(1, 2, 2, projection='3d')
xx=np.array(old_norm_X[:,0]).flatten()
yy=np.array(old_norm_X[:,1]).flatten()
zz=np.array(old_norm_X[:,2]).flatten()
ax.scatter(xx,yy,zz,s=80,marker=(5,3),c=['r','r','b','b','r','g','b','r','g','b','r','b','r','b','g','g'])
ax.set_title("ppdata colored by data set")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show() 